In [ ]:
import numpy as np

def logistic_function(x):
    """
    Computes the logistic function applied to any value of x.
    Arguments:
        x: scalar or numpy array of any size.
    Returns:
        y: logistic function applied to x.
    """
    y = 1 / (1 + np.exp(-x))
    return y

In [ ]:
def log_loss(y_true, y_pred):
    """
    Computes log loss for true target value y = {0 or 1} and predicted target value y' in {0-1}.
    """
    # Ensure y_pred is clipped to avoid log(0)
    y_pred = np.clip(y_pred, 1e-10, 1 - 1e-10)
    loss = -(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    return loss

In [ ]:
def cost_function(y_true, y_pred):
    """
    Computes log loss for inputs true value (0 or 1) and predicted value (between 0 and 1)
    """
    assert len(y_true) == len(y_pred), "Length mismatch"
    n = len(y_true)
    loss_vec = log_loss(y_true, y_pred)
    cost = np.sum(loss_vec) / n
    return cost

In [ ]:
def costfunction_logreg(X, y, w, b):
    """
    Computes the cost function, given data and model parameters.
    """
    n, d = X.shape
    assert len(y) == n, "Number of observations mismatch"
    assert len(w) == d, "Number of features mismatch"

    # Compute z using np.dot
    z = np.dot(X, w) + b  # Matrix-vector multiplication and adding bias

    # Compute predictions using logistic function
    y_pred = logistic_function(z)

    # Compute the cost
    cost = cost_function(y, y_pred)
    return cost

In [ ]:
def costfunction_logreg(X, y, w, b):
    z = np.dot(X, w) + b
    y_pred = logistic_function(z)
    return cost_function(y, y_pred)
def compute_gradient(X, y, w, b):
    n, d = X.shape
    z = np.dot(X, w) + b
    y_pred = logistic_function(z)

    error = y_pred - y
    grad_w = np.dot(X.T, error) / n
    grad_b = np.sum(error) / n
    return grad_w, grad_b
X = np.array([[10,20],[-10,10]])
y = np.array([1,0])
w = np.array([0.5,1.5])
b = 1
grad_w, grad_b = compute_gradient(X,y,w,b)
print("Gradient test passed.")


Gradient test passed.


In [ ]:
def compute_gradient(X, y, w, b):
    """
    Computes gradients of the cost function with respect to model parameters.
    """
    n, d = X.shape
    assert len(y) == n, f"Expected y to have {n} elements"
    assert len(w) == d, f"Expected w to have {d} elements"

    # Compute predictions
    z = np.dot(X, w) + b
    y_pred = logistic_function(z)

    # Compute gradients
    error = y_pred - y
    grad_w = np.dot(X.T, error) / n  # Gradient w.r.t weights
    grad_b = np.sum(error) / n       # Gradient w.r.t bias

    return grad_w, grad_b

In [ ]:
def gradient_descent(X, y, w, b, alpha, n_iter, show_cost=False, show_params=True):
    """
    Implements batch gradient descent to optimize logistic regression parameters.
    """
    n, d = X.shape
    cost_history = []
    params_history = []

    for i in range(n_iter):
        # Compute gradients
        grad_w, grad_b = compute_gradient(X, y, w, b)

        # Update weights and bias
        w -= alpha * grad_w
        b -= alpha * grad_b

        # Compute cost
        cost = costfunction_logreg(X, y, w, b)

        # Store history
        cost_history.append(cost)
        params_history.append((w.copy(), b))

        # Optional printing
        if show_cost and (i % 100 == 0 or i == n_iter - 1):
            print(f"Iteration {i}: Cost = {cost:.6f}")
        if show_params and (i % 100 == 0 or i == n_iter - 1):
            print(f"Iteration {i}: w = {w}, b = {b:.6f}")

    return w, b, cost_history, params_history

In [ ]:
def prediction(X, w, b, threshold=0.5):
    """
    Predicts binary outcomes for given input features.
    """
    # Compute predicted probabilities
    z = np.dot(X, w) + b
    y_test_prob = logistic_function(z)

    # Classify based on threshold
    y_pred = (y_test_prob >= threshold).astype(int)

    return y_pred

In [ ]:
def evaluate_classification(y_true, y_pred):
    """
    Computes confusion matrix, precision, recall, and F1-score.
    """
    # Initialize confusion matrix components
    TP = np.sum((y_true == 1) & (y_pred == 1))  # True Positives
    TN = np.sum((y_true == 0) & (y_pred == 0))  # True Negatives
    FP = np.sum((y_true == 0) & (y_pred == 1))  # False Positives
    FN = np.sum((y_true == 1) & (y_pred == 0))  # False Negatives

    # Confusion matrix
    confusion_matrix = np.array([[TN, FP],
                                 [FN, TP]])

    # Precision, recall, and F1-score
    precision = TP / (TP + FP) if (TP + FP) > 0.0 else 0.0
    recall = TP / (TP + FN) if (TP + FN) > 0.0 else 0.0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

    # Metrics dictionary
    metrics = {
        "confusion_matrix": confusion_matrix,
        "precision": precision,
        "recall": recall,
        "f1_score": f1_score
    }
    return metrics